Load the data and modify it as needed

In [6]:
import pandas as pd
data = pd.read_csv('training_data/training_data/ED/train.csv')

#BERT takes one input sentence, so we need to concatenate the claim and the evidence
data['text'] = data['Claim'] + " [SEP] " + data['Evidence']


Tokenization and labelling

In [7]:
from transformers import BertTokenizer
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# tokenization
inputs = tokenizer(list(data['text']), padding="max_length", truncation=True, max_length=512, return_tensors="pt")

#put labels in correct format
labels = torch.tensor(data['label'])


Load the Pre-trained BERT Model

In [8]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Setting up training enviornment

In [9]:
from torch.optim import AdamW
from transformers import get_scheduler

#Adam weighted optimizer
#model.parameters() gets weights and biases, lr == learning rate
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(inputs['input_ids'])
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
